# __Potential Wind Capacity Analysis for California__

## __Import Necessary Packages__

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from IPython.display import display, clear_output
from selenium.webdriver.common.keys import Keys
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
from selenium import webdriver
from bs4 import BeautifulSoup
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
import numpy as np
import googlemaps
import requests
import folium
import pickle
import geopy
import json
import time
import re

## __Store & Retrieve Data using Pickle__
Functions to save and retrieve data of a variable

In [2]:
def pickle_save(path, data):
    file_path = path
    
    # Open the file in binary mode
    with open(file_path, 'wb') as file:
        # Serialize and write the variable to the file
        pickle.dump(data, file)
    
    print(f"The data {path} has been saved successfully.")

In [3]:
def pickle_load(path):
    file_path = path
    
    with open(file_path, 'rb') as file:
        # Deserialize and retrieve the variable from the file
        loaded_data = pickle.load(file)
    
    print(f"The data {path} has been loaded successfully.")

    return loaded_data

## __Wind Speed History for California__
Obtain the wind speed data zipcode-wise for California.

### Zipcodes from California State Geoportal
The data for zipcodes is from <https://gis.data.ca.gov/datasets/CDEGIS::california-zip-codes/explore>. This dataset also has area, population density etc.

In [4]:
# Path to Datset
file_path = "../reference_dataset/ca_zipcode_data.csv"

# Load the CSV file into a pandas DataFrame
zipcode_data = pd.read_csv(file_path, encoding='latin-1')
    
# Print Shape of Loaded Data
print(f"File with shape {zipcode_data.shape} loaded successfully.")

File with shape (1721, 9) loaded successfully.


In [5]:
# Print Columns of the Loaded Data
print(f"Loaded Data has Columns: {list(zipcode_data.columns)}")

Loaded Data has Columns: ['ï»¿OBJECTID', 'Zip Code', 'Post Office Name', 'State', 'Pop 2017', 'Pop SqMile 2017', 'SqMile', 'Shape__Area', 'Shape__Length']


In [6]:
# Extract only the zipcodes
zipcodes = list(zipcode_data['Zip Code'])

# Filter out valid 5 digit zipcodes
zipcodes = [zipcodes[i] for i in range(len(zipcodes)) if(zipcodes[i] > 90000)]

In [7]:
# Create a dataframe with zipcode and names for those places
city_names = zipcode_data[['Zip Code','Post Office Name']]
city_names = city_names.rename(columns={'Zip Code':'zipcode', 'Post Office Name':'City'})

In [8]:
# Create a dataframe with zipcode and the population density and area of that zipcode
pop_density = zipcode_data[['Zip Code','Pop SqMile 2017', 'SqMile']]
pop_density = pop_density.rename(columns={'Zip Code':'zipcode', 'Pop SqMile 2017':'PopDensity', 'SqMile':'Area'})

### Wind Speed History from National Weather Service
The website <https://www.weather.gov/> has been scraped to get the wind speed history for zipcodes.

In [9]:
# Function to get coordinates of a zipcode
def get_coord(zipcode):
    geolocator = geopy.Nominatim(user_agent="us")
    search_loc = f'{zipcode}, USA'
    location = geolocator.geocode(search_loc)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [10]:
# Function to find link to wind history page of a zipcode
def get_his_link(zipcode):
    
    # get coordinates for zipcode
    latitude, longitude = get_coord(zipcode)
    coord = f'{zipcode}: {latitude}, {longitude}'
    
    # get data from NWS for respective coordinates
    site = f'https://forecast.weather.gov/MapClick.php?lat={latitude}&lon={longitude}'
    source = requests.get(site).text
    soup = BeautifulSoup(source, 'lxml')
    
    try:
        # get standard weather details
        location = soup.find('h2', class_='panel-title').text
        forecast = soup.find('p', class_='myforecast-current').text
        temp = soup.find('p', class_='myforecast-current-lrg').text
        tempc = soup.find('p', class_='myforecast-current-sm').text
    
        curr_conditions = soup.find(attrs={'id': 'current_conditions_detail'})
        wind = curr_conditions.find(text='Wind Speed')
        b_tag = wind.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')

        print(coord)
        '''
        print(f'Weather for {location}')
        print(forecast)
        print(temp)
        print(tempc)
        print(next_td_tag.contents[0])
        '''
    
    except AttributeError:
        print('Invalid Location')
        time.sleep(2)

    # get link to weather history
    curr_conditions_extra = soup.find(attrs={'class': 'current-conditions-extra'})
    three_day_his = curr_conditions_extra.find('a', text='3 Day History').get("href")
    
    return three_day_his

In [11]:
# Function to get windspeed history
def get_wind_speed(zipcode, driver):
    # get weather history link
    three_day_his = get_his_link(zipcode)
    # request data based on retrieved link
    his_source = requests.get(three_day_his).text
    his_soup = BeautifulSoup(his_source, 'lxml')
    his_data = his_soup.find('table', attrs={'id': 'OBS_DATA'})

    # Load Selenium
    try:
        driver.get(three_day_his)
    except TimeoutException:
        driver.execute_script("window.stop();")

    # Wait for details to load
    time.sleep(0.1)
    headers = []

    # If details not loaded, loop until details loaded or till timeout
    start = time.time()
    while True:
        end = time.time()
        if (end - start > 25):
            break 
        try:
            # get weather history data
            his_source = BeautifulSoup(driver.page_source, 'lxml')
            his_data = his_source.find('table', attrs={'id': 'OBS_DATA'})
            header_data = his_data.find('thead')
            headers = header_data.find_all('th')
            table_data = his_data.find('tbody')
            break
        except:
            pass
            
    headers = [header.text for header in headers]

    # Extract only windspeed history
    df = pd.read_html(str(his_data))[0]
    wind_his = df['WindSpeed(mph)']
    wind_his_data = [zipcode, three_day_his, wind_his]
    
    return wind_his_data

In [12]:
# Function to find windspeed history for all zipcodes
def get_all_zipcode_data(zipcodes):
    wind_his_data = []
    
    # create a selenium driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_page_load_timeout(10)
    
    # Iterate through all zipcodes and fetch data
    for zipcode in tqdm(zipcodes):
        try:
            wind_his_data.append(get_wind_speed(zipcode, driver))
            clear_output(wait=True)
        except:
            wind_his_data.append([zipcode, None, None])
    
    return wind_his_data

In [ ]:
# Get wind speed history for all zipcodes
wind_his_data = get_all_zipcode_data(zipcodes)

### Weather Data from Visual Crossing (API)
The [visual crossing API](https://www.visualcrossing.com/weather-api) provides 1000 free data retrievals per day. However, each year has 365 days of wind speed data, it is not feasible to fetch data for all ~1.7k zipcodes in the free tier.

In [23]:
# Range of data to be fetched
start_date = '2023-1-1'
end_date = '2023-1-2'

# zipcode for which data needs to be fetched
location = '90290'

# Use visual crossing API key
key = 'USE_VISUAL_CROSSING_KEY_HERE'

# Query String
query = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup=us&key={key}&include=obs'

# Get data
response = requests.request("GET", query)
if response.status_code!=200:
  print('Unexpected Status code: ', response.status_code) 

# Parse the results as JSON
jsonData = response.json()

In [24]:
# Show fetched data
jsonData

{'queryCost': 2,
 'latitude': 34.101447,
 'longitude': -118.59933,
 'resolvedAddress': '90290, USA',
 'address': '90290',
 'timezone': 'America/Los_Angeles',
 'tzoffset': -8.0,
 'days': [{'datetime': '2023-01-01',
   'datetimeEpoch': 1672560000,
   'tempmax': 59.4,
   'tempmin': 51.0,
   'temp': 54.8,
   'feelslikemax': 59.4,
   'feelslikemin': 51.0,
   'feelslike': 54.8,
   'dew': 44.7,
   'humidity': 69.5,
   'precip': 0.236,
   'precipprob': 100.0,
   'precipcover': 29.17,
   'preciptype': ['rain'],
   'snow': None,
   'snowdepth': None,
   'windgust': 29.8,
   'windspeed': 11.8,
   'winddir': 294.1,
   'pressure': 1007.3,
   'cloudcover': 48.5,
   'visibility': 9.6,
   'solarradiation': 58.7,
   'solarenergy': 4.9,
   'uvindex': 3.0,
   'sunrise': '07:00:15',
   'sunriseEpoch': 1672585215,
   'sunset': '16:55:56',
   'sunsetEpoch': 1672620956,
   'moonphase': 0.32,
   'conditions': 'Rain, Partially cloudy',
   'description': 'Partly cloudy throughout the day with rain.',
   'icon':

In [29]:
# Save data fetched for a month
monthSample = jsonData
pickle_save('../obtained_dataset/monthSample.pickle', monthSample)

The variable 'data' has been saved successfully.


### Wind Speed History from Weather Spark (Scraping)
[Weather Spark](https://weatherspark.com/countries/US/CA) also has a paid API but it also has a website from which the wind speed data can be fetched using scraping.

In [13]:
# Function to get wind speed data from weather spark
def get_history_spark(zipcode, driver):
    search_query = f'CA ({zipcode})'
    print(search_query)

    # enter the zipcode in searchbar and search
    search_element = driver.find_element("xpath", '//*[@id="LiveSearch-NavBar-InputContainer"]/input[@class="LiveSearch-field LiveSearch-NavBar-field form-control"]')
    search_element.send_keys(search_query)
    search_element.send_keys(Keys.ENTER)
    button_element = driver.find_element("xpath", '//*[@class="btn btn-primary"]')
    button_element.click()

    # wait for page to load and retrieve the element with windspeed data
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    wind_section = soup.find('a', attrs={'id':'Figures-WindSpeed'})
    wind_table = wind_section.findNext(attrs={'class':'MonthlyAveragesTable-outer_container'})
    wind_table = wind_table.find('table')

    # make a dataframe with the retrieved data
    df = pd.read_html(str(wind_table))[0]
    
    return df, driver

In [14]:
# Function to iterate through the given zipcodes and get the respective wind speed data
def get_all_history_spark(zipcodes):
    wind_his_data = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_page_load_timeout(10)
    
    base_link = 'https://weatherspark.com/countries/US/CA'
    try:
        driver.get(base_link)
    except TimeoutException:
        driver.execute_script("window.stop();")

    # Fetch data and store in list
    for zipcode in zipcodes:
        try:
            data, driver = get_history_spark(zipcode, driver)
            wind_his_data.append([zipcode, data])
            time.sleep(0.1)
        except:
            wind_his_data.append([zipcode, None])
            
    return wind_his_data

__Fetch data for zip codes in groups of 10 and store it in order to maintain the data even if there is any issue.__

In [16]:
spark_wind_his_data = []

# Assign starting index and steps for zipcodes
ind = 0
end = 0
step = 10
zip_size = len(zipcodes)
pbar = tqdm(total=zip_size)

# Call the funtion for zipcodes with steps of 10 in order to store details for every 10 zipcodes
while (ind < zip_size):
    # condition when the last set of zipcodes are selected
    if(ind + step > zip_size):
        end = zip_size
    else:
        end = ind + step
    spark_wind_his_data += get_all_history_spark(zipcodes[ind:end])
    # update index to new set of zipcodes
    ind = end
    clear_output(wait=True)
    pbar.update(step)
pbar.close()

1690it [2:03:01,  4.37s/it]                                                                                            


In [15]:
# Function to fetch details of zipcodes which have not been fetched previosuly
def get_all_rem_history_spark(inds, wind_his_data):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_page_load_timeout(10)
    
    base_link = 'https://weatherspark.com/countries/US/CA'
    try:
        driver.get(base_link)
    except TimeoutException:
        driver.execute_script("window.stop();")

    # Iterate through all the indices for zipcodes not been fetched
    for i in inds:
        try:
            # call function for zipcode for specific index
            data, driver = get_history_spark(wind_his_data[i][0], driver)
            wind_his_data[i][1] = data
            time.sleep(0.1)
        except:
            wind_his_data[i][1] = None
            
    return wind_his_data

In [16]:
# Function to check for zipcode with data not fetched and store the indices for those zipcode
def check_none(his_data):
    inds = []
    count = 0
    for i in range(len(his_data)):
        if(isinstance(his_data[i][1], pd.DataFrame)):
            continue
        elif(his_data[i][1] == None):
            inds.append(i)
            count += 1
    return inds

In [138]:
# Get the indices
empty_inds = check_none(spark_wind_his_data)

In [139]:
# Check how many zipcodes data has not been fetched
len(empty_inds)

1

In [140]:
ind = 0
end = 0
step = 10
zip_size = len(empty_inds)
pbar = tqdm(total=zip_size)

# Call the funtion for zipcodes with steps of 10 in order to store details for every 10 zipcodes
while (ind < zip_size):
    if(ind + step > zip_size):
        end = zip_size
    else:
        end = ind + step
    spark_wind_his_data = get_all_rem_history_spark(empty_inds[ind:end], spark_wind_his_data)
    # update index to new set of zipcodes
    ind = end
    clear_output(wait=True)
    pbar.update(step)
pbar.close()

10it [00:17,  1.76s/it]                                                                                                


In [157]:
# Sample of data fetched
spark_wind_his_data[751]

[93314,
                                           Unnamed: 0  Jan  Feb  Mar  Apr  May  \
 0  JanFebMarAprMayJunJulAugSepOctNovDec Wind Spee...  NaN  NaN  NaN  NaN  NaN   
 1                                                NaN  Jan  Feb  Mar  Apr  May   
 2                                   Wind Speed (mph)  4.3  4.7  5.2  5.9  6.6   
 
    Jun  Jul  Aug  Sep  Oct  Nov  Dec  
 0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
 1  Jun  Jul  Aug  Sep  Oct  Nov  Dec  
 2  6.7  5.9  5.2  4.8  4.5  4.3  4.4  ]

In [144]:
# Save the data that has been fetched
pickle_save('../obtained_dataset/spark_wind_his_data.pickle', spark_wind_his_data)

The variable 'data' has been saved successfully.


In [19]:
# Load the data that has been saved
spark_wind_his_data = pickle_load('../obtained_dataset/spark_wind_his_data.pickle')

The data ../obtained_dataset/spark_wind_his_data.pickle has been loaded successfully.


## __Process Retrieved Data__
Clean and process data from different sources for analysis.

### Process Data from Weather Spark

In [17]:
# Function to get only the necessary data from the retrieved data
def get_structured_data(wind_his_data):
    structured_wind_data = []
    columns = ['zipcode', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Mean', 'Max']

    # Iterate through all the rows and fetch only the row with the data
    for data in wind_his_data:
        wind_data = data[1].iloc[2]
        wind_data = wind_data[1:]
        wind_data = [float(i) for i in wind_data]
        
        # Calculate the mean annual wind speed
        mean = sum(wind_data) / len(wind_data)

        # Get max monthly wind speed for a year
        max_spd = max(wind_data)
        curr_data = [data[0]] + wind_data + [mean] + [max_spd]
        structured_wind_data.append(curr_data)
        
    structured_wind_data = pd.DataFrame(structured_wind_data, columns=columns)

    return structured_wind_data

In [20]:
# Get cleaned data
all_structured_wind_data = get_structured_data(spark_wind_his_data)

In [21]:
# Since the zipcodes which are close by might be in the same city, it is possible they have the same data
# Sample of cleaned data
all_structured_wind_data

,zipcode,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Mean,Max
0,90001,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4
1,90002,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4
2,90003,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4
3,90004,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4
4,90005,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,96146,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2
1685,96148,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2
1686,96150,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2
1687,96161,6.0,6.2,6.5,6.4,5.9,5.6,5.1,4.9,5.1,5.3,5.9,6.2,5.758333,6.5


In [20]:
pickle_save('../obtained_dataset/all_structured_wind_data.pickle', all_structured_wind_data)

The data all_structured_wind_data.pickle has been saved successfully.


In [22]:
all_structured_wind_data = pickle_load('../obtained_dataset/all_structured_wind_data.pickle')

The data ../obtained_dataset/all_structured_wind_data.pickle has been loaded successfully.


### Threshold on Data from Weather Spark

Since a minimum cut-off wind speed is required for a turbine, a threshold is required to find zipcodes in which potential wind farms can be established

In [24]:
# Function to apply threshold on Mean annual wind speed
def threshold_wind_data(wind_his_data, threshold):
    wind_his_data = wind_his_data.query(f'Mean > {threshold}')
    return wind_his_data

In [25]:
# Get data with threshold applied
all_threshold_wind_data = threshold_wind_data(all_structured_wind_data, 7)

### Wind Turbine
The data for all wind turbines in the US is from https://eerscmap.usgs.gov/uswtdb/. This dataset also has coordinates, capacity etc.

In [27]:
# Path to Datset
file_path = "../reference_dataset/US_wind_turbine_data.csv"

# Load the CSV file into a pandas DataFrame
wind_turbines = pd.read_csv(file_path, encoding='latin-1')
    
# Print Shape of Loaded Data
print(f"File with shape {wind_turbines.shape} loaded successfully.")

File with shape (73352, 27) loaded successfully.


In [28]:
# Sample of read data
wind_turbines

,ï»¿case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_rsa,t_ttlh,retrofit,retrofit_year,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat
0,3123985,40-098167,2021-WTW-7918-OE,NaN,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,17671.46,180.1,0,NaN,3,3,1/4/2023,Digital Globe,-99.787033,36.501724
1,3123544,40-097504,2021-WTW-7909-OE,NaN,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,17671.46,180.1,0,NaN,3,3,1/4/2023,Digital Globe,-99.725624,36.437126
2,3123887,40-097762,2021-WTW-7895-OE,NaN,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,17671.46,180.1,0,NaN,3,3,1/4/2023,Digital Globe,-99.769722,36.444931
3,3123765,40-097546,2021-WTW-7863-OE,NaN,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,17671.46,180.1,0,NaN,3,3,1/4/2023,Digital Globe,-99.807060,36.513935
4,3123814,40-097528,2021-WTW-7897-OE,NaN,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,17671.46,180.1,0,NaN,3,3,1/4/2023,Digital Globe,-99.758476,36.444984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73347,3124478,40-098732,2021-WTW-794-OE,NaN,NaN,OK,Woods County,40151,unknown Woods County,2021.0,...,NaN,NaN,0,NaN,1,3,10/17/2022,Digital Globe,-98.633461,36.500687
73348,3053232,36-120351,2015-WTE-5788-OE,NaN,NaN,NY,Wyoming County,36121,unknown Wyoming County,2016.0,...,NaN,NaN,0,NaN,1,3,5/20/2017,Digital Globe,-78.187935,42.740818
73349,3040944,48-024978,2011-WTW-352-OE,41364.0,NaN,TX,Young County,48503,unknown Young County 1,2011.0,...,NaN,NaN,0,NaN,1,3,7/16/2018,Digital Globe,-98.551094,33.093292
73350,3055918,08-072237,2015-WTW-9995-OE,NaN,NaN,CO,Yuma County,8125,unknown Yuma County,2016.0,...,NaN,NaN,0,NaN,1,3,5/17/2017,Digital Globe,-102.716949,40.037548


In [30]:
# Filter only the wind turbines in california
wind_turbines = wind_turbines.loc[wind_turbines['t_state'] == 'CA']
# Get only the caolumns which are necessary
wind_turbines = wind_turbines[['ï»¿case_id', 't_state', 't_county', 'p_name', 'p_cap', 't_cap', 'xlong', 'ylat']]

In [31]:
# Get all the unique project names in california
all_projects = list(wind_turbines['p_name'].unique())

In [32]:
# Store the coordinates of all turbines project-wise along with the project capacity
project_coords = []
# Iterate through all the project names
for project in all_projects:
    turbine_coords = []
    # if the project name is the same as current project names, then store the coordinates
    for i in range(len(wind_turbines[wind_turbines['p_name'] == project])):
        turbine_coords.append({'lat': wind_turbines[wind_turbines['p_name'] == project].iloc[i]['ylat'], 'lon': wind_turbines[wind_turbines['p_name'] == project].iloc[i]['xlong']})
    cap = None
    for i in range(len(wind_turbines[wind_turbines['p_name'] == project])):
        if not np.isnan(wind_turbines[wind_turbines['p_name'] == project].iloc[i]['p_cap']):
            cap = wind_turbines[wind_turbines['p_name'] == project].iloc[i]['p_cap']
    
    project_coords.append([turbine_coords, cap])

In [34]:
# Function to get zipcode for a project from it's coordinates
def zipcode_from_coord_googleAPI(project_coords):
    gmaps = googlemaps.Client(key='USE_GOOGLE_MAPS_API_KEY')
    zipcodes = []
    rem_coords = []
    
    # Iterate through all projects
    for turbine_coords in project_coords:
        zipcode = None
        extra_coord = [turbine_coords[0][0]]
        
        # Iterate through all the turbine cordinates of a project
        for coord in turbine_coords[0]:
            # Look up the zipcode with reverse geocoding
            reverse_geocode_result = gmaps.reverse_geocode((coord['lat'], coord['lon']))
            # Store one zipcode from the results obtained
            for result in reverse_geocode_result:
                for data in result['address_components']:
                    if('postal_code' in data['types']):
                        zipcode = int(data['short_name'])
                        
            # If a zipcode has been obtained then store the obtained zipcode along with the project capacity
            if(zipcode != None):
                zipcodes.append([zipcode, turbine_coords[1]])
                extra_coord = []
                # Break from the loop once a zipcode has been obtained for that project
                break

        # concatenate the coordinate if it has not been obtained
        rem_coords += extra_coord
        
    return zipcodes, rem_coords

In [ ]:
# Get zipcode for each project based on the coordinates
turbine_zipcodes, rem_coords = zipcode_from_coord_googleAPI(project_coords)

In [105]:
# Create a dataframe from the zipcodes and capacity from previous step
all_proj_details = pd.DataFrame(turbine_zipcodes, columns=['zipcode', 'InstalledCapacity'])

# There may be multiple projects in one zipcode, hence we can find the wind capacity zipcode-wise
all_proj_details = all_proj_details.groupby(['zipcode'])['InstalledCapacity'].sum()
all_proj_details = all_proj_details.to_frame()

In [94]:
# Total Wind capacity of the US based on the data obtained
all_proj_details['Installed_Capacity'].sum()

6061.445

In [122]:
# Merge all the dataframe data and fill data which is not available
merged_data = pd.merge(all_structured_wind_data, pop_density, on="zipcode")
merged_data = pd.merge(merged_data, all_proj_details, on="zipcode", how='left')
merged_data['InstalledCapacity'] = merged_data['InstalledCapacity'].fillna(0)

In [38]:
# Sum all the installed wind capacity in CA and the area in which the installed capacity is more than 0
Current_US_Capacity = merged_data[merged_data['InstalledCapacity'] > 0]
Installed_Capacity = Current_US_Capacity['InstalledCapacity'].sum()
Installed_Area = Current_US_Capacity['Area'].sum()
print(f"California has a capacity of {Installed_Capacity} in an area of {Installed_Area}")

California has a capacity of 6061.445 in an area of 7462.22


__This dataframe shows all the windfarms currently established in california along with other important details such as area and population density of that zipcode__

In [39]:
Current_US_Capacity

,zipcode,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Mean,Max,PopDensity,Area,InstalledCapacity
275,91739,7.3,7.0,6.7,6.8,6.4,6.1,5.7,5.4,5.3,5.8,6.6,7.3,6.366667,7.3,2979.63,13.65,1.000
310,91905,8.2,7.7,7.2,7.1,6.5,5.9,5.5,5.3,5.4,6.1,7.3,8.3,6.708333,8.3,13.28,130.91,182.950
423,92230,7.7,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,6.2,7.0,7.9,7.000000,7.9,157.81,19.15,8.050
429,92240,7.7,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,6.2,7.0,7.9,7.000000,7.9,519.78,71.99,80.630
430,92241,6.6,7.0,7.4,8.1,8.0,7.6,6.7,6.0,6.0,6.2,6.4,6.6,6.883333,8.1,86.64,107.92,20.000
442,92259,7.7,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,6.2,7.0,7.9,7.000000,7.9,2.16,179.53,42.660
444,92262,6.2,6.6,7.0,7.7,7.6,7.3,6.4,5.7,5.7,5.9,6.0,6.2,6.525000,7.7,706.10,40.63,292.275
454,92282,7.7,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,6.2,7.0,7.9,7.000000,7.9,16.90,75.10,357.160
461,92307,7.7,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,6.2,7.0,7.9,7.000000,7.9,189.79,211.46,3.000
465,92311,7.6,7.8,8.1,8.6,8.7,8.3,7.4,6.7,6.6,6.7,7.2,7.7,7.616667,8.7,78.45,421.30,1.500


In [139]:
pickle_save('../obtained_dataset/all_merged_data.pickle', merged_data)

The data all_merged_data.pickle has been saved successfully.


In [36]:
merged_data = pickle_load('../obtained_dataset/all_merged_data.pickle')

The data ../obtained_dataset/all_merged_data.pickle has been loaded successfully.


__From the current installed capacity and the area in which that respective capacity is being held, a factor can be calculated for CA with the metric MW/Sq.Mi (Mega Watts per Sq. Mile). If this factor is multiplied with the area (Sq. Miles) of a zipcode, the potential wind capacity (MW) for that zipcode can be found.__

In [40]:
installed_capacity = []
# Find the factor explained above, for each of the zipcode with established windfarms
for i in range(len(Current_US_Capacity)):
    installed_capacity.append(Current_US_Capacity.iloc[i]['InstalledCapacity'] / Current_US_Capacity.iloc[i]['Area'])
    
# Calculate the average for the factor
sum(installed_capacity) / len(installed_capacity)

1.072014404258363

## __Visualize Data__

### Read GeoJSON for California

In [41]:
# Open GeoJSON with zipcode boundaries which has been obtained from CA state geoportal
with open('../reference_dataset/ca_zipcode_geo.json') as f:
   zipcode_plot_data = json.load(f)

### Plot Mean Annual Wind Speed from Weather Spark

In [42]:
# Get only the necessary data from the dataframe
mean_windspeed_data = all_structured_wind_data[['zipcode','Mean']]
# Merge the data with city names to display in the map
mean_windspeed_data = pd.merge(mean_windspeed_data, city_names, on="zipcode")

In [45]:
# Create a base map which is zoomed into california
m = folium.Map(location=[36.7783, -119.4179], zoom_start=6)

# Create a Choropleth layer using the mean data
cp = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Mean Category',
    data=mean_windspeed_data,
    columns=['zipcode', 'Mean'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Mean',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    show=True
).add_to(m)

# Add Mean Wind Speed data and Zipcode name to the geojson to display it in tooltip
for s in cp.geojson.data['features']:
    mean_value = list(mean_windspeed_data.loc[mean_windspeed_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'Mean'])
    city_value = list(mean_windspeed_data.loc[mean_windspeed_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['Mean'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['Mean'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'Mean', 'City'], aliases=['Zipcode:','Mean Windspeed:', 'Place:']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

m.save("../plots/mean_annual_wind_speed.html")

### Plot Threshold (7) Data from Weather Spark

In [46]:
# Get only the necessary data from the dataframe
df = all_threshold_wind_data[['zipcode','Mean']]
# Merge the data with city names to display in the map
df = pd.merge(df, city_names, on="zipcode")

In [48]:
# Create a base map which is zoomed into california
m = folium.Map(location=[36.7783, -119.4179], zoom_start=6)

# Create a Choropleth layer using the mean data
cp = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Mean Category',
    data=df,
    columns=['zipcode', 'Mean'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Mean',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    show=True
).add_to(m)

# Add Mean Wind Speed data and Zipcode name to the geojson to display it in tooltip
for s in cp.geojson.data['features']:
    mean_value = list(df.loc[df['zipcode'] == int(s['properties']['ZIP_CODE']), 'Mean'])
    city_value = list(df.loc[df['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['Mean'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['Mean'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'Mean', 'City'], aliases=['Zipcode:','Mean Windspeed:', 'Place:']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

m.save("../plots/mean_threshold_annual_wind_speed.html")

### Plot Land Cost for each Zipcode

In [49]:
# Read data with land price which has been obtained from loopnet in the other program
all_merged_data_price = pd.read_csv('../obtained_dataset/all_zipcodes_land_cost.csv', encoding='latin-1')

In [50]:
all_merged_data_price = all_merged_data_price.rename(columns={'Zipcode':'zipcode', 'Average Price per Plot Size Unit':'AvgPlotCost'})

In [51]:
land_price_data = pd.merge(all_merged_data_price, city_names, on="zipcode")
# Change metric from Dollars per Acre to Million Dollars per Sq. Mile
land_price_data['AvgPlotCost'] = land_price_data['AvgPlotCost'] * 640 / 1000000

In [52]:
# Sample of data that has been read
land_price_data

,zipcode,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Mean,Max,PopDensity,Area,InstalledCapacity,AvgPlotCost,City
0,90001,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4,16970.82,3.53,0.0,5796.12864,Los Angeles
1,90002,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4,17281.61,3.10,0.0,2388.94976,Los Angeles
2,90003,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4,20503.42,3.51,0.0,5350.54976,Los Angeles
3,90004,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4,20508.65,3.12,0.0,13262.05440,Los Angeles
4,90005,8.3,7.8,7.3,7.2,6.5,5.9,5.4,5.2,5.4,6.2,7.4,8.4,6.750000,8.4,26823.08,1.56,0.0,12190.05568,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,96146,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2,149.42,7.71,0.0,2295.65184,Olympic Valley
1685,96148,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2,991.33,1.50,0.0,4756.00000,Tahoe Vista
1686,96150,6.6,6.9,7.2,7.1,6.6,6.2,5.6,5.4,5.6,5.9,6.6,6.9,6.383333,7.2,151.87,217.28,0.0,4339.33632,South Lake Tahoe
1687,96161,6.0,6.2,6.5,6.4,5.9,5.6,5.1,4.9,5.1,5.3,5.9,6.2,5.758333,6.5,99.57,194.99,0.0,3055.32096,Truckee


In [54]:
# Create a base map which is zoomed into california
m = folium.Map(location=[36.7783, -119.4179], zoom_start=6)

# Create a Choropleth layer using the prepared data
cp = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Mean Category',
    data=land_price_data,
    columns=['zipcode', 'AvgPlotCost'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Mean',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    show=True
).add_to(m)

# Add Land cost and Zipcode name to the geojson to display it in tooltip
for s in cp.geojson.data['features']:
    mean_value = list(land_price_data.loc[land_price_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'AvgPlotCost'])
    city_value = list(land_price_data.loc[land_price_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['AvgPlotCost'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['AvgPlotCost'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'AvgPlotCost', 'City'], aliases=['Zipcode:','Land Cost:', 'Place:']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

m.save("../plots/land_cost_plot.html")

### Plot Wind Turbines

In [55]:
# Read US wind turbine data in geojson format to plot all the turbines
file_path = "../reference_dataset/US_wind_turbine_data.geojson"
gdf = gpd.read_file(file_path)
gdf = gdf[gdf['t_state'] == 'CA']
gdf.head()

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,xlong,ylat,geometry
60,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764)
61,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791)
62,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744)
304,3025453.0,CA,Alite Wind Farm,2008.0,8.0,24.00,Vestas,V90-3.0,3000.0,80.0,90.0,6361.73,125.0,0.0,NaN,3.0,3.0,-118.343689,35.031494,POINT (-118.34369 35.03149)
305,3025446.0,CA,Alite Wind Farm,2008.0,8.0,24.00,Vestas,V90-3.0,3000.0,80.0,90.0,6361.73,125.0,0.0,NaN,3.0,3.0,-118.345291,35.035194,POINT (-118.34529 35.03519)


In [56]:
# Create a base map which is zoomed into california
m = folium.Map(location=[36.7783, -119.4179], zoom_start=6)

# Create a marker cluster for all turbines
marker_cluster = MarkerCluster().add_to(m)

# Iterate through all the rows and add it to the marker cluster
for idx, row in gdf.iterrows():
    folium.Marker([row.geometry.centroid.y, row.geometry.centroid.x],
                  popup=f"<b>Location:</b> {row['t_state']}<br><b>Capacity (MW):</b> {row['p_cap']}"
                 ).add_to(marker_cluster)
    
m.save("../plots/ca_turbine_plot.html")

### All Plots with Layers

In [57]:
# Read the cleaned data with thresholds applied which has been calculated in the other program
potential_widfarms = pd.read_csv('../obtained_dataset/potential_windfarms_data.csv', encoding='latin-1')
potential_widfarms = potential_widfarms.rename(columns={'Zipcode':'zipcode', 'Average Price per Plot Size Unit':'AvgPlotCost'})
potential_widfarms = pd.merge(potential_widfarms, city_names, on="zipcode")

In [58]:
# Change Cost from Dollars to Millions
potential_widfarms['Total Cost'] = potential_widfarms['Total Cost'] / 1000000
potential_widfarms['Total Cost'] = round(potential_widfarms['Total Cost'], 2)

In [59]:
potential_widfarms

,zipcode,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,...,Max,PopDensity,Area,InstalledCapacity,AvgPlotCost,Potential Capacity,Difference in Capacity,Land required,Total Cost,City
0,91384,8.9,8.4,8.2,8.2,7.5,6.8,6.1,5.8,6.0,...,9.1,159.31,194.17,0.00,22802022400,208.153037,208.153037,0.325239,7686.71,Castaic
1,91390,8.8,8.2,7.9,7.8,7.0,6.4,5.8,5.5,5.6,...,8.9,136.45,158.92,0.00,53329920000,170.364529,170.364529,0.266195,14417.61,Santa Clarita
2,91901,7.6,7.7,7.8,8.3,8.3,7.8,6.9,6.3,6.3,...,8.3,186.74,98.71,0.00,960787660800,105.818542,105.818542,0.165341,158995.61,Alpine
3,91934,7.8,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,...,8.0,30.82,24.92,0.00,10240000000,26.714599,26.714599,0.041742,462.16,Jacumba
4,91935,7.8,7.8,7.7,7.8,7.4,6.6,6.1,5.9,5.9,...,8.0,76.01,125.10,0.00,239894937600,134.109002,134.109002,0.209545,50443.20,Jamul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,95023,6.3,6.6,6.9,7.3,7.8,8.0,7.6,7.1,6.4,...,8.0,87.16,601.29,17.03,2624612480,644.591541,627.561541,0.980565,3389.43,Hollister
138,95620,6.3,6.6,6.9,7.3,7.8,8.0,7.6,7.1,6.4,...,8.0,96.88,229.25,1.00,180845440,245.759302,244.759302,0.382436,387.35,Dixon
139,95695,6.6,6.8,6.8,6.8,6.9,7.0,7.1,6.8,6.1,...,7.1,277.24,142.15,1.50,2273731200,152.386848,150.886848,0.235761,732.21,Woodland
140,96013,4.3,4.5,4.6,4.5,4.4,4.4,4.1,4.0,4.1,...,4.6,13.04,336.58,101.20,341964800,360.818608,259.618608,0.405654,476.22,Burney


In [106]:
# Create a base map which is zoomed into california
m = folium.Map(location=[36.7783, -119.4179], tiles=None, zoom_start=6)

In [107]:
folium.TileLayer('openstreetmap', overlay=True, show=True, control=False).add_to(m)

In [108]:
# Create a Choropleth layer using the mean data
cp = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Mean Wind Speed',
    data=mean_windspeed_data,
    columns=['zipcode', 'Mean'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Mean Annual Wind Speed in Miles per Hour',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    overlay=False,
    show=False
).add_to(m)

# Add Mean Wind Speed data and Zipcode name to the geojson to display it in tooltip
for s in cp.geojson.data['features']:
    mean_value = list(mean_windspeed_data.loc[mean_windspeed_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'Mean'])
    city_value = list(mean_windspeed_data.loc[mean_windspeed_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['Mean'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['Mean'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'Mean', 'City'], aliases=['Zipcode:','Mean Windspeed:', 'Place:']).add_to(cp.geojson)

In [109]:
# Turbine Plot

# Create a marker cluster for all turbines
marker_cluster = MarkerCluster(name='Wind Turbines', show=False).add_to(m)

# Iterate through all the rows and add it to the marker cluster
for idx, row in gdf.iterrows():
    folium.Marker([row.geometry.centroid.y, row.geometry.centroid.x],
                  popup=f"<b>Location:</b> {row['t_state']}<br><b>Capacity (MW):</b> {row['p_cap']}"
                 ).add_to(marker_cluster)

In [110]:
# Land Price Plot

# Create a Choropleth layer using land price
lp = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Land Price',
    data=land_price_data,
    columns=['zipcode', 'AvgPlotCost'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Land Cost per Sq. Mile',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    overlay=False,
    show=False
).add_to(m)

# Add Land cost and Zipcode name to the geojson to display it in tooltip
for s in lp.geojson.data['features']:
    mean_value = list(land_price_data.loc[land_price_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'AvgPlotCost'])
    city_value = list(land_price_data.loc[land_price_data['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['AvgPlotCost'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['AvgPlotCost'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'AvgPlotCost', 'City'], aliases=['Zipcode:','Land Cost:', 'Place:']).add_to(lp.geojson)

In [111]:
# Potential Windfarms

# Create a Choropleth layer using Total cost
pw = folium.Choropleth(
    geo_data=zipcode_plot_data,
    name='Potential Windfarms',
    data=potential_widfarms,
    columns=['zipcode', 'Total Cost'],
    key_on='properties.ZIP_CODE', # Adjust based on your data structure
    fill_color='YlOrRd',
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    fill_opacity=0.8,
    fill_gradient=True,
    legend_name='Cost of Potential Windfarms in Millions',
    highlight=True,
    line_color='black',
    line_weight=0.1,
    overlay=False,
    show=True
).add_to(m)

# Add Total cost and Zipcode name to the geojson to display it in tooltip
for s in pw.geojson.data['features']:
    mean_value = list(potential_widfarms.loc[potential_widfarms['zipcode'] == int(s['properties']['ZIP_CODE']), 'Total Cost'])
    city_value = list(potential_widfarms.loc[potential_widfarms['zipcode'] == int(s['properties']['ZIP_CODE']), 'City'])
    if(len(mean_value) > 0):
        s['properties']['Total Cost'] = round(mean_value[0], 4)
        s['properties']['City'] = city_value[0]
    else:
        s['properties']['Total Cost'] = 0
        s['properties']['City'] = 'N/A'

folium.GeoJsonTooltip(fields=['ZIP_CODE', 'Total Cost', 'City'], aliases=['Zipcode:','Cost in Millions:', 'Place:']).add_to(pw.geojson)

In [112]:
folium.LayerControl().add_to(m)

m.save("../plots/final_plot.html")